# MorseGraph Example 2: Data-Driven Dynamics

This notebook demonstrates how to compute a Morse graph from a dataset of input-output pairs `(X, Y)`. This is useful when the dynamics are not known from a function or ODE, but are given by data (e.g., from a simulation or experiment).

We will use the `BoxMapData` dynamics class. For this example, we first generate a dataset `(X, Y)` from a known map (the Henon map) so we can see how the data-driven approach works.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import the necessary components from the MorseGraph library
from morsegraph.grids import UniformGrid
from morsegraph.dynamics import BoxMapData
from morsegraph.core import Model
from morsegraph.analysis import compute_morse_graph
from morsegraph.plot import plot_morse_graph, plot_morse_sets

## 1. Generate Sample Data

First, let's create a dataset. We'll use the classic Henon map to generate points. We sample random points `X` in a box and compute their images `Y` under the map.

In [ ]:
def henon_map(x, a=1.4, b=0.3):
    """ Standard Henon map. """
    x_next = 1 - a * x[:, 0]**2 + x[:, 1]
    y_next = b * x[:, 0]
    return np.column_stack([x_next, y_next])

# Define the domain and number of sample points
lower_bounds = np.array([-1.5, -0.4])
upper_bounds = np.array([1.5, 0.4])
num_points = 5000

# Generate random points X and their images Y
X = np.random.uniform(low=lower_bounds, high=upper_bounds, size=(num_points, 2))
Y = henon_map(X)

# Plot the data to see what it looks like
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], s=5, c='blue', label='Original Points (X)')
plt.scatter(Y[:, 0], Y[:, 1], s=5, c='red', label='Mapped Points (Y)')
plt.title("Henon Map Dataset")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

## 2. Set up the Morse Graph Computation

Now, we define the grid and the dynamics model. Instead of `BoxMapFunction`, we'll use `BoxMapData`, passing it our generated `X` and `Y` arrays.

In [ ]:
# Define the grid parameters
subdivisions = 12
domain = np.array([[-1.5, 1.5], [-0.4, 0.4]])

# 1. Create the dynamics object from our data
# We add a small bloat_factor to ensure outer approximation.
dynamics = BoxMapData(X, Y, bloat_factor=0.1)

# 2. Create the grid
grid = UniformGrid(subdivisions, domain)

# 3. Create the model which connects the grid and dynamics
model = Model(grid, dynamics)

# 4. Compute the state transition graph (map graph)
# This can take a moment as it evaluates the map for each box in the grid.
print("Computing map graph...")
map_graph = model.compute_map_graph()
print("Map graph computed.")

# 5. Compute the Morse graph from the map graph
morse_graph = compute_morse_graph(map_graph)

## 3. Visualize the Results

Finally, we can plot the Morse graph and the Morse sets on the grid. The Morse graph shows the connectivity between recurrent components (the Morse sets), and the plot of the sets shows where they are located in the state space.

In [ ]:
# Plot the Morse graph (condensation graph)
fig, ax = plt.subplots()
plot_morse_graph(morse_graph, ax=ax)
ax.set_title("Morse Graph")
plt.show()

# Plot the Morse sets on the grid
fig, ax = plt.subplots(figsize=(10, 5))
plot_morse_sets(morse_graph, grid, ax=ax)
ax.set_title("Morse Sets")
plt.show()